**In this example we will examine the motion of a particle undergoing anharmonic oscillations.**

Oscillating systems are ubiquitous in physics.  We can often approximate that small-amplitude oscillations are described by simple harmonic oscillators with potential energy $V(x)=\frac{1}{2}kx^2$, where $k$ is the spring constant, because potential wells are generally quadratic about their minima in accordance with Taylor's theorem.  However, for larger amplitudes, we can no longer assume that the potential energy is quadratic and hence we have an **anharmonic oscillator**.

Suppose particle of mass m moves in a *quartic* potential energy

$$V(x) = \frac{1}{2}kx^2 + ax^3 + bx^4,$$

where $k$ and $b$ are positive constants and $a$ is a constant that may be of either sign.

To avoid confusion, we shall assume that all quantities are in SI units.  (In "real life" it is often useful to choose appropriate units that give sensible numbers; e.g., in quantum chemistry and condensed matter physics we often use Hartree atomic units in which the numerical values of the electron mass $m_e$, magnitude of electronic charge $|e|$, permittivity $4\pi\epsilon_0$ and reduced Planck constant $\hbar$ all take value 1.)

We introduce a class for anharmonic oscillators, holding the potential parameters and including methods to return the potential and the force due to the potential, as well as a method to plot the potential.

**Can you add a method to plot the force as a function of particle position?**

When we model this anharmonic oscillator, we assume that we still have just a single minimum at $x=0$ in the potential.  **Can you derive a condition on the cubic coefficient $a$ to ensure that no additional stationary points are introduced?**  Hints: (i) at stationary points the force on the particle is zero; (ii) use the discriminant of a quadratic to ensure that no additional stationary points are introduced.  **Can you add some code to raise a value error in the __init__ method if parameter values that would lead to additional stationary points are supplied.**

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import textwrap as tw

# The default font sizes, etc., for matplotlib are far too small.  I don't know why.
mpl.rc('font',size=21)
mpl.rc('lines',linewidth=4,marker='',markersize=6)
mpl.rc('axes',linewidth=3,grid=False)
mpl.rc('xtick.major',size=10,width=3)
mpl.rc('xtick.minor',visible=True,size=5,width=3)
mpl.rc('ytick.major',size=10,width=3)
mpl.rc('ytick.minor',visible=True,size=5,width=3)

class ao:
    """Anharmonic oscillator of mass m in potential well V(x) = k.x^2/2 + a.x^3 + b.x^4."""
    
    def __init__(self,m=1.0,k=1.0,a=0.1,b=0.1):
        if m<=0.0:
            raise ValueError("Mass m should be positive.")
        self.m=m # Mass
        if k<=0.0:
            raise ValueError("Spring constant k should be positive.")
        self.k=k # "Spring constant"
        self.a=a # Cubic coefficient
        if b<0.0:
            raise ValueError("Quartic coefficient b should be non-negative.")
        self.b=b # Quartic coefficient.

    def __str__(self):
        return tw.fill("Anharmonic oscillator, mass m={0:.15g}, spring constant k={1:.15g}, "
                       "cubic anharmonicity a={2:.15g}, quartic anharmonicity b={3:.15g}"
                       .format(self.m,self.k,self.a,self.b))

    def V(self,x):
        """Return a quartic potential energy function V(x) = k.x^2/2 + a.x^3 + b.x^4."""
# Factorising the expression makes it more efficient to evaluate.
        return x*x*(0.5*self.k+x*(self.a+x*self.b))

    def F(self,x):
        """Return the force on a particle in a quartic potential, F(x) = -V'(x) = -k.x - 3.a.x^2 - 4.b.x^3."""
        return -x*(self.k+x*(3.0*self.a+x*4.0*self.b))
    
    def plot_potential(self):
        """Plot the potential together with the harmonic part of the potential."""
        fig=plt.figure()
        ax=fig.add_subplot(1,1,1)
        ax.set_xlabel(r'$x$ (m)')
        ax.set_ylabel(r'$V(x)$ (J)')
        xx=np.arange(-3.0,3.0,0.1)  # Array of x values to plot.
        VV=self.V(xx) # Function V can take a NumPy array xx and return an array of corresponding potential values.
        VV_harmonic=0.5*self.k*xx**2 # Harmonic potential values, for comparison.
        ax.plot(xx,VV,label="anharmonic")
        ax.plot(xx,VV_harmonic,label="harmonic")
        ax.legend()
        plt.tight_layout()
        plt.show()

s=ao() # Default anharmonic oscillator.
s.plot_potential()

Total energy (kinetic plus potential) is conserved throughout motion, i.e.,

$$\frac{1}{2}mv^2 + V(x) = E = {\rm constant},$$
  
where $v=dx/dt$ is velocity and $x$ is particle position.

Hence velocity is

$$v = \pm \sqrt{2[E - V(x)] / m}.$$
 
The turning points $x_l$ and $x_r$ occur when $v=0$, i.e., where $V(x)=E$.

The time taken to get from one turning point to the other is

$$\frac{T}{2} = \int_{x_l}^{x_r} \frac{dx}{v}.$$

Double this to find the oscillation period $T$.

In [ ]:
from scipy import integrate

class ao_solver(ao):
    """Extension of anharmonic oscillator class containing methods to find the period by integration."""
    
    def find_tps_nr(self,xinit,E):
        """Use Newton-Raphson to solve V(x)-E=0, giving the turning point x."""
        xp=xinit-1.0
        x=xinit
        i=0
        while abs(x-xp)>1.E-15:
            i+=1
            if i>10000:
                raise RuntimeError("Newton-Raphson iteration has failed to converge.")
            xp=x
            try:
                x=x+(self.V(x)-E)/self.F(x) # NB, F(x)=-dV/dx.
            except ZeroDivisionError:
                x=1.E-8*np.sign(xinit)
        return x

    def find_tps(self,E):
        """Find the left and right turning points (i.e., solutions of V(x)=E)."""
        xrh=np.sqrt(2.0*E/self.k) # First guess: (+/-)sqrt(2.E/k), which is correct for harmonic well.
        xr=self.find_tps_nr(xrh,E) ; xl=self.find_tps_nr(-xrh,E) # Use Newton-Raphson to find the turning points.
        if xr-xl<1.E-12*abs(xr): # Check that xl and xr and the distinct left and right turning points.
            raise RuntimeError("Have not found distinct turning points in correct order.")
        return xl,xr

    def reciprocal_speed(self,x,E):
        """Return one over the speed of the particle at position x with total energy E."""
        return np.sqrt(self.m/(2.0*(max(E-self.V(x),0.0))))

    def period(self,E):
        """Evaluate the period of the particle's oscillation as twice the time taken to get from
        the left-hand to the right-hand turning point.  This time is Int_xl^xr dx/|v(x)|, where |v(x)|
        is the speed of the particle.  Use a canned numerical integration routine."""
        xl,xr=self.find_tps(E)
        I=integrate.quad(self.reciprocal_speed,xl,xr,args=E)
        return 2.0*I[0],2.0*I[1]
    
    def plot_period_against_E(self):
        """Plot oscillation frequency against energy."""
        EE=np.arange(0.0001,10.0,0.1) # Avoid numerical problems calculating the oscillation period at E=0.
        TT=[] ; errTT=[]
        for E in EE:
            T=self.period(E)
            TT.append(T[0]) ; errTT.append(T[1])
        fig=plt.figure()
        ax=fig.add_subplot(1,1,1)
        ax.set_xlabel(r'$E$ (J)')
        ax.set_ylabel(r'$T$ (s)')
        ax.errorbar(EE,TT,errTT)
        plt.tight_layout()
        plt.show()

s=ao_solver() # Default anharmonic oscillator.
s.plot_period_against_E()

Now we will simulate Newton's laws of motion for the particle undergoing anharmonic oscillations.

We chop up time into a series of small "time steps" $dt$.  Acceleration is rate of change of velocity; so velocity approximately changes by acceleration times $dt$ over the time step.  Likewise, position changes by velocity times $dt$ over a time step.

**Can you add a method to evaluate the total energy for a particular position $x$ and velocity $v$, and then plot the energy against time during the particle's motion?**

**Can you add code that examines the simulation results and evaluates the oscillation period?  Do your results agree with the orbital period obtained in the previous cell?**

In [ ]:
class ao_simulator(ao):
    """Extend the anharmonic oscillator class to include some methods for simulating Newton's laws."""

    def simulate(self,xinit,vinit,dt_over_Tharmonic=0.001,Nharmonic_periods=4.0):
        """Carry out a numerical simulation of the motion of the particle in the anharmonic potential
        by numerically solving Newton's laws of motion.  Plot position and velocity against time."""
        harmonic_period=2.0*np.pi*np.sqrt(self.m/self.k) # Period in absence of anharmonicity.
        dt=dt_over_Tharmonic*harmonic_period # Time step.
        Nsteps=int(Nharmonic_periods*harmonic_period/dt) # Number of time steps - enough for a few periods.
        t=0.0 ; x=xinit ; v=vinit # Initial time, position and velocity.
        tt=[] ; xx=[] ; vv=[]
        for _i in range(Nsteps):
            tt.append(t) ; xx.append(x) ; vv.append(v)
            x,v=self.new_pos_vel(x,v,self.F,self.m,dt)
            t+=dt
        return tt,xx,vv

    @staticmethod
    def new_pos_vel(x,v,F,m,dt):
        """Return updated position and velocity after time step dt.  Current position and velocity
        are x and v.  Function F evaluates the force.  m is the particle mass and dt is the time step.
        The Euler-Cromer method is used."""
        vnew=v+F(x)/m*dt # New velocity is old velocity plus old acceleration times dt.
        xnew=x+vnew*dt # Euler-Cromer: new position is old position plus *new* velocity times dt. 
        return xnew,vnew

s=ao_simulator() # Default anharmonic oscillator.
tt,xx,vv=s.simulate(0.0,1.0) # Simulate a few periods, starting at x=0 with velocity 1.0 m/s.

fig=plt.figure(figsize=(8,12))
ax=fig.add_subplot(2,1,1)
ax.set_xlabel(r'$t$ (s)')
ax.set_ylabel(r'$x(t)$ (m)')
ax.plot(tt,xx)
bx=fig.add_subplot(2,1,2)
bx.set_xlabel(r'$t$ (s)')
bx.set_ylabel(r'$v(t)$ (m/s)')
bx.plot(tt,vv)
plt.tight_layout()
plt.show()